# DataTalks.Club MLOps Zoomcamp - Homework 1

In [1]:
import pandas as pd

In [2]:
!pip install pyarrow

In [3]:
fpath = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
df = pd.read_parquet(fpath)

In [4]:
len(df.columns)

19

## Q1
2023 Jan dataset has 19 columns

In [5]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [6]:
df['duration_td'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

In [7]:
df['duration_td'].head()

0   0 days 00:08:26
1   0 days 00:06:19
2   0 days 00:12:45
3   0 days 00:09:37
4   0 days 00:10:50
Name: duration_td, dtype: timedelta64[ns]

In [8]:
df.duration_td.std()

Timedelta('0 days 00:42:35.661074517')

In [9]:
df.duration = df.duration_td.apply(lambda dtd: dtd.total_seconds()/60)

/tmp/ipykernel_72927/3375784082.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.duration = df.duration_td.apply(lambda dtd: dtd.total_seconds()/60)


In [10]:
df.duration.head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration_td, dtype: float64

In [11]:
df.duration.std()

42.594351241920904

## Q2
In 2023 Jan, standard duration of trip duration was 42.59 minutes  

In [12]:
df.duration.isna().any()

False

In [16]:
df_cl = df.loc[(df.duration >= 1) & (df.duration <= 60)]

In [17]:
df_cl.shape[0]/df.shape[0]*100

98.1220282212598

## Q3
In Jan 2023 rides, 98.12% of the durations were between 1 and 60 minutes, inclusive.